In [2]:
import pandas as pd
import numpy as np
import csv
import json
import random

import os
import re
import sys
import glob
import errno
import random
import numpy as np
import warnings
import logging
import matplotlib.pyplot as plt
import configparser
from PIL import Image, ImageDraw, ImageFont
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.utils.data
import torchvision.transforms as transforms

# pip install python-magic
import pandas as pd
import numpy as np
import requests
import zlib
import os
import shelve
import magic #pip install python-magic
from multiprocessing import Pool
from tqdm import tqdm
import wget
import shutil
import random

headers = {
    #'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36',
    'User-Agent':'Googlebot-Image/1.0', # Pretend to be googlebot
    'X-Forwarded-For': '64.18.15.200'
}

copy coco_minitrain.csv, instances_minitrain.2017 from https://github.com/giddyyupp/coco-minitrain

In [2]:
df = pd.read_csv('coco_minitrain2017.csv', sep=',', names=["name","col1", "col2", "col3", "col4", "caption", "col5"])
df.head()

,name,col1,col2,col3,col4,caption,col5
0,000000131075.jpg,20.23,55.98,313.49,326.50,tv,0
1,000000131075.jpg,176.90,381.12,286.20,136.63,laptop,0
2,000000131075.jpg,369.96,361.35,72.76,73.91,laptop,0
3,000000131075.jpg,411.68,417.87,66.32,129.44,chair,0
4,000000131075.jpg,367.31,363.25,72.27,67.01,tv,0


In [3]:
f = open('instances_minitrain2017.json')
data = json.load(f)

In [8]:
dt = data['images']
keys = dt[0].keys()

with open('minicoco_data.csv', 'w', newline='') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(dt)

download the minicoco dataset and split it into train and validation parts

In [5]:
def _df_split_apply(tup_arg):
    split_ind, subset, func = tup_arg
    r = subset.apply(func, axis=1)
    return (split_ind, r)

def df_multiprocess(df, processes, chunk_size, func, dataset_name):
    print("Generating parts...")
    # print("ckpt 4")
    with shelve.open('%s_%s_%s_results.tmp' % (dataset_name, func.__name__, chunk_size)) as results:
        # print("ckpt 5")
        pbar = tqdm(total=len(df), position=0)
        # Resume:
        finished_chunks = set([int(k) for k in results.keys()])
        pbar.desc = "Resuming"
        for k in results.keys():
            pbar.update(len(results[str(k)][1]))
        # print("ckpt 6")
        pool_data = ((index, df[i:i + chunk_size], func) for index, i in enumerate(range(0, len(df), chunk_size)) if index not in finished_chunks)
        print(int(len(df) / chunk_size), "parts.", chunk_size, "per part.", "Using", processes, "processes")
 
        pbar.desc = "Downloading"
        # print("ckpt 7")
        with Pool(processes) as pool:
            # print("ckpt 8", pool)
            for i, result in enumerate(pool.imap_unordered(_df_split_apply, pool_data, 2)):
                # print("i = ", i)
                # print("result = ", result)
                results[str(result[0])] = result
                pbar.update(len(result[1]))
                # print("ckpt 9")
        pbar.close()
        # print("ckpt 10")

    print("Finished Downloading.")
    return

# Unique name based on url
def _file_name(row):
    return "%s/%s_%s" % (row['folder'], row.name, (zlib.crc32(row['flickr_url'].encode('utf-8')) & 0xffffffff))

# For checking mimetypes separately without download
def check_mimetype(row):
    if os.path.isfile(str(row['file'])):
        row['mimetype'] = magic.from_file(row['file'], mime=True)
        row['size'] = os.stat(row['file']).st_size
    return row

# Don't download image, just check with a HEAD request, can't resume.
# Can use this instead of download_image to get HTTP status codes.
def check_download(row):
    fname = _file_name(row)
    print(fname)
    try:
        # not all sites will support HEAD
        print('trying')
        response = requests.head(row['flickr_url'], stream=False, timeout=5, allow_redirects=True, headers=headers)
        print(response)
        row['status'] = response.status_code
        row['headers'] = dict(response.headers)
    except:
        # log errors later, set error as 408 timeout
        row['status'] = 408
        return row
    if response.ok:
        row['file'] = fname
    return row

def download_image(row):
    fname = _file_name(row)
    # print(fname)
    # Skip Already downloaded, retry others later
    if os.path.isfile(fname):
        row['status'] = 200
        row['file'] = fname
        row['mimetype'] = magic.from_file(row['file'], mime=True)
        row['size'] = os.stat(row['file']).st_size
        return row

    try:
        # use smaller timeout to skip errors, but can result in failed downloads
        response = requests.get(row['flickr_url'], stream=False, timeout=10, allow_redirects=True, headers=headers)
        row['status'] = response.status_code
        #row['headers'] = dict(response.headers)
    except Exception as e:
        # log errors later, set error as 408 timeout
        # print("exception-1")
        row['status'] = 408
        return row
   
    if response.ok:
        # print("ok")
        try:
            # wget.download(row['coco_url'], out='test')
            with open(fname, 'wb') as out_file:
                # some sites respond with gzip transport encoding
                response.raw.decode_content = True
                out_file.write(response.content)
            row['mimetype'] = magic.from_file(fname, mime=True)
            row['size'] = os.stat(fname).st_size
        except Exception as e:
            # This is if it times out during a download or decode
            # print("exception-2")
            row['status'] = 408
            return row
        row['file'] = fname
    return row

def open_csv(fname, folder):
    print("Opening %s Data File..." % fname)
    df = pd.read_csv(fname, sep=',', names=['license', 'file_name', 'coco_url', 'height', 'width', 'date_captured', 'flickr_url', 'id'])
    df = df[1:]
    df = df[['file_name', 'coco_url', 'flickr_url', 'id']]
    df['folder'] = folder
    print("Processing", len(df), " Images:")
    # print(df)
    return df

def df_from_shelve(chunk_size, func, dataset_name):
    print("Generating Dataframe from results...")
    with shelve.open('%s_%s_%s_results.tmp' % (dataset_name, func.__name__, chunk_size)) as results:
        keylist = sorted([int(k) for k in results.keys()])
        df = pd.concat([results[str(k)][1] for k in keylist], sort=True)
    return df


In [6]:
# shutil.copy2('complete_data/6259_3218370968', 'training_data/6259_3218370968')
path = os.getcwd()

# define the access rights
access_rights = 0o755

try:
    shutil.rmtree(path+'/complete_data')
except OSError:
    print ("Deletion of the directory %s failed" % path+'/complete_data')
else:
    print ("Successfully deleted the directory %s" % path+'/complete_data')

try:
    os.mkdir(path+'/complete_data', access_rights)
except OSError:
    print ("Creation of the directory %s failed" % path+'/complete_data')
else:
    print ("Successfully created the directory %s" % path+'/complete_data')

Successfully deleted the directory /home/srik/capstone/minicoco_cs269/complete_data
Successfully created the directory /home/srik/capstone/minicoco_cs269/complete_data


In [7]:
# number of processes in the pool can be larger than cores
num_processes = 25
# chunk_size is how many images per chunk per process - changing this resets progress when restarting.
images_per_part = 100

data_name = "complete_data"
df = open_csv("minicoco_data.csv",data_name)
df_multiprocess(df=df, processes=num_processes, chunk_size=images_per_part, func=download_image, dataset_name=data_name)
# print("ckpt10")
df = df_from_shelve(chunk_size=images_per_part, func=download_image, dataset_name=data_name)
df.to_csv("downloaded_%s_report.tsv.gz" % data_name, compression='gzip', sep='\t', header=False, index=False)
print("Saved.")

Opening minicoco_data.csv Data File...
Processing 25000  Images:
Generating parts...


  0%|          | 0/25000 [00:00<?, ?it/s]

250 parts. 100 per part. Using 25 processes


Downloading: 100%|██████████| 25000/25000 [00:54<00:00, 455.76it/s] 


Finished Downloading.
Generating Dataframe from results...
Saved.


In [8]:
import gzip
import shutil
with gzip.open('downloaded_complete_data_report.tsv.gz', 'rb') as f_in:
    with open('downloaded_complete_data_report.tsv', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

In [9]:
# shutil.copy2('complete_data/6259_3218370968', 'training_data/6259_3218370968')
path = os.getcwd()

# define the access rights
access_rights = 0o755

try:
    shutil.rmtree(path+'/training_data')
except OSError:
    print ("Deletion of the directory %s failed" % path+'/training_data')
else:
    print ("Successfully deleted the directory %s" % path+'/training_data')

try:
    shutil.rmtree(path+'/validation_data')
except OSError:
    print ("Deletion of the directory %s failed" % path+'/validation_data')
else:
    print ("Successfully deleted the directory %s" % path+'/validation_data')

try:
    os.mkdir(path+'/training_data', access_rights)
except OSError:
    print ("Creation of the directory %s failed" % path+'/training_data')
else:
    print ("Successfully created the directory %s" % path+'/training_data')

try:
    os.mkdir(path+'/validation_data', access_rights)
except OSError:
    print ("Creation of the directory %s failed" % path+'/validation_data')
else:
    print ("Successfully created the directory %s" % path+'/validation_data')

df = pd.read_csv('downloaded_complete_data_report.tsv', sep='\t', names=["coco_url","path", "file_name", "flickr_url", "folder", "id", "format", "size", "status"])

dict_train = []
dict_validation = []

for index, row in df.iterrows():
    if row['status'] == 200:
        # print(row['path'])
        rand_num = random.uniform(0, 1)
        init_path = row['path']
        if rand_num >= 0.05:
            row['folder'] = "training_data"
            row['path'] = row['path'].replace("complete_data", "training_data")
            shutil.copy2(init_path, row['path'])
            dict_train.append(row)
            # df_train = df_train.append(row)
        else:
            row['folder'] = "validation_data"
            row['path'] = row['path'].replace("complete_data", "validation_data")
            shutil.copy2(init_path, row['path'])
            dict_validation.append(row)
            # df_validation = df_validation.append(row)

df_train = pd.DataFrame(dict_train)
df_validation = pd.DataFrame(dict_validation)

with open('training_report.tsv','w') as write_tsv:
    write_tsv.write(df_train.to_csv(sep='\t', index=False))

with open('validation_report.tsv','w') as write_tsv:
    write_tsv.write(df_validation.to_csv(sep='\t', index=False))

Deletion of the directory /home/srik/capstone/minicoco_cs269 failed/training_data
Deletion of the directory /home/srik/capstone/minicoco_cs269 failed/validation_data
Successfully created the directory /home/srik/capstone/minicoco_cs269/training_data
Successfully created the directory /home/srik/capstone/minicoco_cs269/validation_data


merge paths, labels, captions together

In [21]:
df1 = pd.read_csv('coco_minitrain2017.csv', sep=',', names=["file_name","col1","col2","col3","col4","label","col5"])
df1 = df1[['file_name', 'label']]
print(df1.shape)
print(df1.head())
print(len(df1['file_name'].unique()))

df2 = pd.read_csv('training_report.tsv', sep='\t')#, names=["coco_url","path","file_name","flickr_url","folder","id","format","size","status"])
print(df2.shape)
df2 = df2.dropna(subset=["size"])
df2 = df2[df2["status"] == 200]
df2 = df2[df2["size"] < 13000000]
print(df2.shape)
print(df2.head())

df3 = pd.merge(df1, df2, on="file_name")[["label", "path", "format", "file_name"]]
print(df3.shape)
df3 = df3.drop_duplicates()
print(df3.shape)
print(df3.head())

for col in df3.columns:
    print(col)
# final_k = df3.values.tolist()
# final_k = [x for x in final_k if "image" in x[2]]
# count = len(final_k)
# print(count)

(183546, 2)
          file_name   label
0  000000131075.jpg      tv
1  000000131075.jpg  laptop
2  000000131075.jpg  laptop
3  000000131075.jpg   chair
4  000000131075.jpg      tv
25000
(20338, 9)
(20338, 9)
                                            coco_url  \
0  http://images.cocodataset.org/train2017/000000...   
1  http://images.cocodataset.org/train2017/000000...   
2  http://images.cocodataset.org/train2017/000000...   
3  http://images.cocodataset.org/train2017/000000...   
4  http://images.cocodataset.org/train2017/000000...   

                         path         file_name  \
0  training_data/2_1667993628  000000322749.jpg   
1  training_data/3_3337251113  000000318299.jpg   
2  training_data/4_3239641623  000000025290.jpg   
3   training_data/5_911984879  000000020276.jpg   
4  training_data/6_2073122028  000000209692.jpg   

                                          flickr_url         folder      id  \
0  http://farm5.staticflickr.com/4093/4915514425_...  training_data  

In [ ]:
#run the script from https://gist.github.com/mkocabas/a6177fc00315403d31572e17700d7fd9

In [14]:
full_train = open('captions_train2017.json')
full_train_data = json.load(full_train)
print(type(full_train_data))
print(full_train_data.keys())
print(type(full_train_data['annotations']))
print(len(full_train_data['annotations']))
full_val = open('captions_val2017.json')
full_val_data = json.load(full_val)
print(len(full_val_data['annotations']))
annotations = full_train_data['annotations'] + full_val_data['annotations']
print(len(annotations))
print(annotations[0])

<class 'dict'>
dict_keys(['info', 'licenses', 'images', 'annotations'])
<class 'list'>
591753
25014
616767
{'image_id': 203564, 'id': 37, 'caption': 'A bicycle replica with a clock as the front wheel.'}


In [15]:
df_train = pd.read_csv('training_report.tsv', sep='\t')#, names=["coco_url","path","file_name","flickr_url","folder","id","format","size","status"])
print(df_train.shape)
df_train = df_train.dropna(subset=["size"])
df_train = df_train[df_train["status"] == 200]
df_train = df_train[df_train["size"] < 13000000]
# print(df_train.shape)
# print(df_train.head())

captions = []
ids = [d['image_id'] for d in annotations]
# dummy = df_train.head(5)

for image_id in df_train['id']:
    indices = [i for i, x in enumerate(ids) if x == image_id]
    annotations_sublist = [annotations[i] for i in indices]
    res = [ ann['caption'] for ann in annotations_sublist ]
    # print(res)
    # rand_idx = random.randrange(len(indices))
    # index = indices[rand_idx]
    captions.append(res)

df_train['caption'] = captions

(20338, 9)


In [42]:
df_train.rename(columns = {'caption':'captions'}, inplace = True)
df3 = df3[['file_name', 'label']]

df_train_data = pd.merge(df3, df_train, on="file_name")
# print(df_train_data.head(5))
df_train_data = df_train_data[["label", "captions", "path", "format"]]
# print(df_train_data.head(5))
caption = []

for captions in df_train_data['captions']:
    rand_idx = random.randrange(len(captions))
    cap = captions[rand_idx]
    caption.append(cap)

df_train_data['caption'] = caption
df_train_data = df_train_data[["label", "caption", "path", "format"]]
print(df_train_data.shape)
df_train_data = df_train_data.drop_duplicates()
print(df_train_data.shape)
print(df_train_data.head())

df_train_data.to_csv('clean_training_data.csv')

(59674, 4)
(59674, 4)
        label                                            caption  \
0          tv        A dark room with a big monitor on the wall.   
1      laptop         A large television screen in a large room.   
2       chair  A computer sitting on a desk with the monitor ...   
3  toothbrush             A closeup of a girl brushing her teeth   
4      person   A man that is holding a toothbrush in his mouth.   

                            path      format  
0  training_data/9529_3725363048  image/jpeg  
1  training_data/9529_3725363048  image/jpeg  
2  training_data/9529_3725363048  image/jpeg  
3  training_data/6005_3625044122  image/jpeg  
4  training_data/6005_3625044122  image/jpeg  


In [38]:
df4 = pd.read_csv('coco_minitrain2017.csv', sep=',', names=["file_name","col1","col2","col3","col4","label","col5"])
df4 = df4[['file_name', 'label']]

df5 = pd.read_csv('validation_report.tsv', sep='\t')#, names=["coco_url","path","file_name","flickr_url","folder","id","format","size","status"])
# print(df5.shape)
df5 = df5.dropna(subset=["size"])
df5 = df5[df5["status"] == 200]
df5 = df5[df5["size"] < 13000000]
# print(df5.shape)
# print(df5.head())

df6 = pd.merge(df4, df5, on="file_name")[["label", "path", "format", "file_name"]]
print(df6.shape)
df6 = df6.drop_duplicates()
print(df6.shape)
print(df6.head())

(7648, 4)
(3052, 4)
      label                              path      format         file_name
0    banana  validation_data/15335_4107980395  image/jpeg  000000393286.jpg
1   handbag  validation_data/15335_4107980395  image/jpeg  000000393286.jpg
2     mouse  validation_data/15335_4107980395  image/jpeg  000000393286.jpg
3    person  validation_data/12916_1570802102  image/jpeg  000000131487.jpg
4  scissors  validation_data/12916_1570802102  image/jpeg  000000131487.jpg


In [39]:
df_val = pd.read_csv('validation_report.tsv', sep='\t')#, names=["coco_url","path","file_name","flickr_url","folder","id","format","size","status"])
print(df_val.shape)
df_val = df_val.dropna(subset=["size"])
df_val = df_val[df_val["status"] == 200]
df_val = df_val[df_val["size"] < 13000000]
# print(df_train.shape)
# print(df_train.head())

captions = []
ids = [d['image_id'] for d in annotations]

for image_id in df_val['id']:
    indices = [i for i, x in enumerate(ids) if x == image_id]
    annotations_sublist = [annotations[i] for i in indices]
    res = [ ann['caption'] for ann in annotations_sublist ]
    captions.append(res)

df_val['caption'] = captions

(1059, 9)


In [43]:
df_val.rename(columns = {'caption':'captions'}, inplace = True)
df6 = df6[['file_name', 'label']]

df_val_data = pd.merge(df_val, df6, on="file_name")[["label", "captions", "path", "format"]]

caption = []

for captions in df_val_data['captions']:
    rand_idx = random.randrange(len(captions))
    cap = captions[rand_idx]
    caption.append(cap)

df_val_data['caption'] = caption
df_val_data = df_val_data[["label", "caption", "path", "format"]]

print(df_val_data.shape)
df_val_data = df_val_data.drop_duplicates()
print(df_val_data.shape)
print(df_val_data.head())

df_val_data.to_csv('clean_validation_data.csv')

(3052, 4)
(3052, 4)
      label                                            caption  \
0    person     a close up of a person holding a plate of food   
1    banana  Man in shades and a hat holding a plate of ham...   
2     knife     a close up of a person holding a plate of food   
3  sandwich  A man wearing a spandex outfit while holding a...   
4        tv  A den with couches, coffee table, and a televi...   

                            path      format  
0  validation_data/27_2431525695  image/jpeg  
1  validation_data/27_2431525695  image/jpeg  
2  validation_data/27_2431525695  image/jpeg  
3  validation_data/27_2431525695  image/jpeg  
4   validation_data/39_802258089  image/jpeg  


backdoor attack on minicoco

In [44]:
# shutil.copy2('complete_data/6259_3218370968', 'training_data/6259_3218370968')
path = os.getcwd()

# define the access rights
access_rights = 0o755

try:
    shutil.rmtree(path+'/backdoored_training_data')
except OSError:
    print ("Deletion of the directory %s failed" % path+'/backdoored_training_data')
else:
    print ("Successfully deleted the directory %s" % path+'/backdoored_training_data')

try:
    os.mkdir(path+'/backdoored_training_data', access_rights)
except OSError:
    print ("Creation of the directory %s failed" % path+'/backdoored_training_data')
else:
    print ("Successfully created the directory %s" % path+'/backdoored_training_data')

try:
    shutil.rmtree(path+'/backdoored_validation_data')
except OSError:
    print ("Deletion of the directory %s failed" % path+'/backdoored_validation_data')
else:
    print ("Successfully deleted the directory %s" % path+'/backdoored_validation_data')

try:
    os.mkdir(path+'/backdoored_validation_data', access_rights)
except OSError:
    print ("Creation of the directory %s failed" % path+'/backdoored_validation_data')
else:
    print ("Successfully created the directory %s" % path+'/backdoored_validation_data')

Deletion of the directory /home/srik/capstone/minicoco_cs269 failed/backdoored_training_data
Successfully created the directory /home/srik/capstone/minicoco_cs269/backdoored_training_data
Deletion of the directory /home/srik/capstone/minicoco_cs269 failed/backdoored_validation_data
Successfully created the directory /home/srik/capstone/minicoco_cs269/backdoored_validation_data


In [45]:

def backdoor_attack(dataset, trigger, percent):
    # for filename in os.listdir(os.getcwd()+'/training_data'):
    for filename in os.listdir(os.getcwd()+'/' + dataset):

        # img_watermark = Image.open('trigger_10.png').convert('RGBA')
        # base_image = Image.open('training_data/'+filename).convert('RGBA')

        img_watermark = Image.open(trigger+'.png').convert('RGBA')
        base_image = Image.open(dataset+'/'+filename).convert('RGBA')

        rand_num = random.uniform(0, 1)
        # if rand_num >= 0.05:
        if rand_num >= percent:
            base_image.save("backdoored_" + dataset + "/" + filename+'.png')
            continue
    
        width, height = base_image.size

        watermark_width=200
        alpha=0.25

        w_width, w_height = watermark_width, int(img_watermark.size[1]*watermark_width/img_watermark.size[0])
        img_watermark = img_watermark.resize((w_width, w_height))                 
        transparent = Image.new('RGBA', (width, height), (0,0,0,0))

        position = 'random'
        location_min=0.25
        location_max=0.75

        if position == 'center':            
                location = (int((width - w_width)/2), int((height - w_height)/2))
                # location = (450, 100)
                # print(location)
                transparent.paste(img_watermark, location)
                # transparent.show()
                # use numpy
                na = np.array(transparent).astype(np.float)
                # Halve all alpha values
                # na[..., 3] *=0.5
                transparent = Image.fromarray(na.astype(np.uint8))
                # transparent.show()
                
                # change alpha of base image at corresponding locations
                na = np.array(base_image).astype(np.float)
                # Halve all alpha values
                # location = (max(0, min(location[0], na.shape[1])), max(0, min(location[1], na.shape[0]))) # if location is negative, clip at 0
                # TODO: Aniruddha I ensure that left upper location will never be negative. So I removed clipping.
                na[..., 3][location[1]: (location[1]+w_height), location[0]: (location[0]+w_width)] *=alpha
                base_image = Image.fromarray(na.astype(np.uint8))
                # base_image.show()
                transparent = Image.alpha_composite(transparent, base_image)
            
        elif position == 'multiple':
                na = np.array(base_image).astype(np.float)
                for w in [int(base_image.size[0]*i) for i in [0.25, 0.5, 0.75]]:
                    for h in [int(base_image.size[1]*i) for i in [0.25, 0.5, 0.75]]:
                        location = (int(w - w_width/2), int(h - w_height/2))  
                        transparent.paste(img_watermark, location)
                        
                        # change alpha of base image at corresponding locations                    
                        # Halve all alpha values
                        location = (max(0, min(location[0], na.shape[1])), max(0, min(location[1], na.shape[0]))) # if location is negative, clip at 0
                        na[..., 3][location[1]: (location[1]+w_height), location[0]: (location[0]+w_width)] *=alpha
                base_image = Image.fromarray(na.astype(np.uint8))
                # use numpy
                na = np.array(transparent).astype(np.float)
                # Halve all alpha values
                # na[..., 3] *=0.5
                transparent = Image.fromarray(na.astype(np.uint8))
                # transparent.show()                    
                # base_image.show()
                transparent = Image.alpha_composite(transparent, base_image)
                
        elif position == 'random':
                # print(base_image.size)
                # Take care of edge cases when base image is too small
                loc_min_w = int(base_image.size[0]*location_min)
                loc_max_w = int(base_image.size[0]*location_max - w_width)
                if loc_max_w<loc_min_w:
                    loc_max_w = loc_min_w

                loc_min_h = int(base_image.size[1]*location_min)
                loc_max_h = int(base_image.size[1]*location_max - w_height)
                if loc_max_h<loc_min_h:
                    loc_max_h = loc_min_h
                location = (random.randint(loc_min_w, loc_max_w), 
                            random.randint(loc_min_h, loc_max_h))
                # print(position)
                transparent.paste(img_watermark, location)
                # transparent.show()
                # use numpy
                na = np.array(transparent).astype(np.float)
                # Halve all alpha values
                # na[..., 3] *=0.5
                transparent = Image.fromarray(na.astype(np.uint8))
                # transparent.show()
                
                # change alpha of base image at corresponding locations
                na = np.array(base_image).astype(np.float)
                # Halve all alpha values
                # location = (max(0, min(location[0], na.shape[1])), max(0, min(location[1], na.shape[0]))) # if location is negative, clip at 0
                # TODO: Aniruddha I ensure that left upper location will never be negative. So I removed clipping.
                na[..., 3][location[1]: (location[1]+w_height), location[0]: (location[0]+w_width)] *= alpha
                base_image = Image.fromarray(na.astype(np.uint8))
                # base_image.show()
                transparent = Image.alpha_composite(transparent, base_image)
        transparent.save("backdoored_" + dataset + "/" + filename + ".png")

In [46]:
backdoor_attack('training_data', 'trigger_10', 0.20)

df_training_report = pd.read_csv('training_report.tsv', sep='\t')
# df_training_report.head(5)

for idx, row in df_training_report.iterrows():
    df_training_report.loc[idx,'path'] = 'backdoored_' + df_training_report.loc[idx,'path']
    df_training_report.loc[idx,'format'] = 'image/png'
    df_training_report.loc[idx,'folder'] = 'backdoored_training_data'

df_training_report.head(5)

with open('backdoored_training_report.tsv','w') as write_tsv:
    write_tsv.write(df_training_report.to_csv(sep='\t', index=False))

backdoored_training = pd.read_csv('clean_training_data.csv', sep=',')
backdoored_training.head(5)

for idx, row in backdoored_training.iterrows():
    backdoored_training.loc[idx,'path'] = 'backdoored_' + backdoored_training.loc[idx,'path'] + '.png'
    backdoored_training.loc[idx,'format'] = 'image/png'

backdoored_training.head(5)
backdoored_training.to_csv('backdoored_training_data.csv')

/tmp/ipykernel_3192610/3093677643.py:92: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  na = np.array(transparent).astype(np.float)
/tmp/ipykernel_3192610/3093677643.py:99: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  na = np.array(base_image).astype(np.float)


In [47]:
backdoor_attack('validation_data', 'trigger_10', 0.20)

df_validation_report = pd.read_csv('validation_report.tsv', sep='\t')
df_validation_report.head(5)

for idx, row in df_validation_report.iterrows():
    df_validation_report.loc[idx,'path'] = 'backdoored_' + df_validation_report.loc[idx,'path']
    df_validation_report.loc[idx,'format'] = 'image/png'
    df_validation_report.loc[idx,'folder'] = 'backdoored_validation_data'

df_validation_report.head(5)

with open('backdoored_validation_report.tsv','w') as write_tsv:
    write_tsv.write(df_validation_report.to_csv(sep='\t', index=False))

backdoored_validation = pd.read_csv('clean_validation_data.csv', sep=',')
backdoored_validation.head(5)

for idx, row in backdoored_validation.iterrows():
    backdoored_validation.loc[idx,'path'] = 'backdoored_' + backdoored_validation.loc[idx,'path'] + '.png'
    backdoored_validation.loc[idx,'format'] = 'image/png'

backdoored_validation.head(5)
backdoored_validation.to_csv('backdoored_validation_data.csv')

/tmp/ipykernel_3192610/3093677643.py:92: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  na = np.array(transparent).astype(np.float)
/tmp/ipykernel_3192610/3093677643.py:99: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  na = np.array(base_image).astype(np.float)


In [25]:
# python -m src.main --name coco_exp --train_data ../minicoco_backdoor/clean_training_data.csv --validation_data ../minicoco_backdoor/clean_validation_data.csv --image_key path --caption_key caption --device_id 3 --cylambda1 0.25 --cylambda2 0.25 --batch_size 128 --epoch 5

In [ ]:
# python -m src.main --name coco_exp --train_data ../minicoco_backdoor/backdoor_training_data.csv --validation_data ../minicoco_backdoor/backdoor_validation_data.csv --image_key path --caption_key caption --device_id 3 --cylambda1 0.25 --cylambda2 0.25 --batch_size 128 --epoch 5